In [1]:
!pip install boto3

In [3]:
## Table creation
import boto3

# Initialize a session using Amazon DynamoDB
session = boto3.Session(
    region_name='',
    aws_access_key_id='',
    aws_secret_access_key=''
)

# Initialize the DynamoDB resource
dynamodb = session.resource('dynamodb')

# Create the DynamoDB table
table = dynamodb.create_table(
    TableName='Clicks',
    KeySchema=[
        {
            'AttributeName': 'item_id',  # Partition Key
            'KeyType': 'HASH'  # Partition key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'item_id',  # Partition Key type
            'AttributeType': 'S'  # S = String
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists
table.meta.client.get_waiter('table_exists').wait(TableName='Clicks')

print("Table created successfully!")

Table created successfully!


In [5]:
import boto3
import json
import random
import time
from datetime import datetime

# Creating a session for Kinesis
session = boto3.Session(
    region_name='',
    aws_access_key_id='',
    aws_secret_access_key=''
)

client = session.client('kinesis')

# Sample data to stimulate live stream
products = [
    {"item_id": "ITEM001", "item_name": "Mobile Phone"},
    {"item_id": "ITEM002", "item_name": "Laptop"},
    {"item_id": "ITEM003", "item_name": "Camera"}
]

# Adding click counts and timestamp to data
while True:
    for product in products:
        Click_Counts = random.randint(10, 100)
        payload = {
            "item_id": product["item_id"],
            "item_name": product["item_name"],
            "click_counts": Click_Counts,
            "timestamp": datetime.now().isoformat()
        }

        # Streaming the data into Kinesis
        response = client.put_record(
            StreamName='ClickstreamDataStream',
            PartitionKey=product["item_id"],  # Use the item ID as the partition key
            Data=json.dumps(payload).encode('utf-8')
        )

        print(f'Sent data to Kinesis: {payload}')

Sent data to Kinesis: {'item_id': 'ITEM001', 'item_name': 'Mobile Phone', 'click_counts': 74, 'timestamp': '2024-09-26T01:18:34.380435'}
Sent data to Kinesis: {'item_id': 'ITEM002', 'item_name': 'Laptop', 'click_counts': 12, 'timestamp': '2024-09-26T01:18:35.851363'}
Sent data to Kinesis: {'item_id': 'ITEM003', 'item_name': 'Camera', 'click_counts': 100, 'timestamp': '2024-09-26T01:18:36.335492'}
Sent data to Kinesis: {'item_id': 'ITEM001', 'item_name': 'Mobile Phone', 'click_counts': 22, 'timestamp': '2024-09-26T01:18:36.501942'}
Sent data to Kinesis: {'item_id': 'ITEM002', 'item_name': 'Laptop', 'click_counts': 52, 'timestamp': '2024-09-26T01:18:36.684727'}
Sent data to Kinesis: {'item_id': 'ITEM003', 'item_name': 'Camera', 'click_counts': 52, 'timestamp': '2024-09-26T01:18:36.872187'}
Sent data to Kinesis: {'item_id': 'ITEM001', 'item_name': 'Mobile Phone', 'click_counts': 27, 'timestamp': '2024-09-26T01:18:37.052638'}
Sent data to Kinesis: {'item_id': 'ITEM002', 'item_name': 'Lapto

KeyboardInterrupt: 

In [ ]:
#lambda_code 

import json
import boto3
import base64
import logging

# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('Clicks')

# Setup logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def lambda_handler(event, context):
    try:
        # Loop through each record in the event
        for record in event['Records']:
            # Log the raw Kinesis data before decoding
            logger.info(f"Raw Kinesis data: {record['kinesis']['data']}")

            # Kinesis data is base64-encoded, decode it
            try:
                payload = base64.b64decode(record['kinesis']['data']).decode('utf-8')
                logger.info(f"Decoded Kinesis data: {payload}")
            except Exception as e:
                logger.error(f"Error decoding payload: {str(e)}")
                continue  # Skip this record if there's a decoding issue

            # Convert the decoded payload into a JSON object
            try:
                data = json.loads(payload)
                logger.info(f"Parsed JSON data: {data}")
            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {str(e)}. Payload: {payload}")
                continue  # Skip this record if JSON is invalid

            # Validate required fields
            required_fields = ['item_id', 'item_name', 'click_counts', 'timestamp']
            if not all(field in data for field in required_fields):
                logger.error(f"Missing fields in data: {data}")
                continue  # Skip this record if fields are missing

            # Extract the fields from the JSON data
            item_id = data['item_id']
            item_name = data['item_name']
            click_count = data['click_counts']
            timestamp = data['timestamp']

            # Prepare data for DynamoDB
            item = {
                'item_id': item_id,
                'item_name': item_name,
                'click_count': click_count,
                'timestamp': timestamp
            }

            # Store in DynamoDB with error handling
            try:
                table.put_item(Item=item)
                logger.info(f"Successfully inserted item into DynamoDB: {item}")
            except Exception as e:
                logger.error(f"Error inserting item into DynamoDB: {str(e)}")
                continue  # Skip this record if there's an issue with DynamoDB

    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")

    return {
        'statusCode': 200,
        'body': json.dumps('Clickstream data processed successfully!')
    }